# Tic Tac Toe Coin

To store state of tic tac toe game on Chia blockchain, we create an outer puzzle that wrap the inner tic tac toe puzzle. The coin puzzle outputs the conditions based on the output from the inner tic tac toe puzzle.

<img src="tic-tac-toe-coin.jpg" alt="Tic Tac Toe Coin" width="600"/>

## Code
### [coin.clsp](./code/coin.clsp)
```lisp
;   MOD                         : the puzzle itself
;   PLAYER_ONE_INFO             : (PK, PUZZLE_HASH), public key and puzzle hash of the first player
;   PLAYER_TWO_INFO             : (PK, PUZZLE_HASH), public key and puzzle hash of the second player
;   CURRIED_TIC_TAC_TOE_PUZZLE  : tic tac toe puzzle with curried-in current board and next player
;   AMOUNT                      : coin amount (odd to be used with singleton top layer)
;   position                    : next play position as inner solution for curried tic tac toe puzzle

(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
     ...
)     
```

## Generate PKs for player one and two
```sh
cdv inspect keys --random | grep public -i
cdv inspect keys --random | grep public -i
Public Key: ab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0
Public Key: 9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc
```


In [1]:
%%bash
chia version
cdv --version
python --version

1.5.0
cdv, version 1.0.8
Python 3.8.10


## 1. Prepare The Game

In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle)

sys.path.insert(0, "./code")
import tic_tac_toe

# load puzzles
tic_tac_toe_puzzle = load_program("./code/tic-tac-toe.clsp", ["./code", "../../../shared"])
coin_puzzle = load_program("./code/coin.clsp", ["./code", "../../../shared"])
terminate_puzzle = load_program("./code/terminate-game.clsp", ["./code", "../../../shared"])

# prepare players info
player_one_pk = bytes.fromhex("ab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0")
player_one_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(G1Element.from_bytes(player_one_pk)).get_tree_hash()
player_two_pk = bytes.fromhex("9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc")
player_two_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(G1Element.from_bytes(player_two_pk)).get_tree_hash()
player_one_info = Program.to([player_one_pk, player_one_hash])
player_two_info = Program.to([player_two_pk, player_two_hash])

player_amount = 1_000
player_fee = 10
p2_amount = player_amount - player_fee
game_amount = player_amount * 2

# prepare terminate-game puzzle
# (mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
curried_terminate_puzzle = terminate_puzzle.curry(
    0,
    player_one_hash,
    player_two_hash,
    p2_amount
)

In [3]:
def sim_play(board, player, position):

    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE AMOUNT position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    conditions = curried_coin_puzzle.run(Program.to([position]))
    coin_puzzle_hash = conditions.at("rfrf")
    print(coin_puzzle_hash)

    print()
    print_program(conditions)
    print()
    board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
    tic_tac_toe.print_board(board)
    print({
         0 : '',
        -1 : 'tie game',
        120: 'x wins',
        113: 'o wins'
    }[board_state])

    if board_state == 0:
        ## verify puzzle
        player = 'x' if player == 'o' else 'o'
        curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
                Program.to(board), 
                Program.to(player)
            )
        curried_coin_puzzle = coin_puzzle.curry(
            coin_puzzle,
            curried_terminate_puzzle,
            player_one_info,
            player_two_info,
            curried_tic_tac_toe_puzzle,
            game_amount)

        # new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
        assert curried_coin_puzzle.get_tree_hash() == coin_puzzle_hash
        return board, player
    return board, None

## 2. Simulate Winning Board
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until `x` wins.

In [4]:
board, player = sim_play([' '] * 9, 'x', 4)
board, player = sim_play(board, player, 0)
board, player = sim_play(board, player, 3)
board, player = sim_play(board, player, 1)
board, player = sim_play(board, player, 5)

a07b1466b1b767622a364818f6c2963100554aada5928c78d550c01a5385d0af22

((50 0xab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0 0xe52d9c508c502347344d8c07ad91cbd6068afc75ff6292f062a09ca381c89e71) (51 0x7b1466b1b767622a364818f6c2963100554aada5928c78d550c01a5385d0af22 2000))

   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


a0e351d2136442e5d98fd930b3e46b156a3c82845f9da4fb449ac34ed36e6fc1c4

((50 0x9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc 0xe3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855) (51 0xe351d2136442e5d98fd930b3e46b156a3c82845f9da4fb449ac34ed36e6fc1c4 2000))

 o |   |   
---+---+---
   | x |   
---+---+---
   |   |   


a059876c007fb7ab0ace9855665dc865c2401ccca6774c7f261af565cf41510f7c

((50 0xab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0 0x084fed08b978af4d7d196a7446a86b58009e636b611db16211b65a9aadff29c5

## 3. Simulate Tie Game
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until the board is not playable.

In [5]:
board, player = sim_play([' '] * 9, 'x', 0)
board, player = sim_play(board, player, 1)
board, player = sim_play(board, player, 2)
board, player = sim_play(board, player, 4)
board, player = sim_play(board, player, 5)
board, player = sim_play(board, player, 6)
board, player = sim_play(board, player, 7)
board, player = sim_play(board, player, 8)
board, player = sim_play(board, player, 3)

a08688dde920940c11c6512284e41acf2b53bb68f06531b46848e5f047a7d9e927

((50 0xab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0 0xe3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855) (51 0x8688dde920940c11c6512284e41acf2b53bb68f06531b46848e5f047a7d9e927 2000))

 x |   |   
---+---+---
   |   |   
---+---+---
   |   |   


a04e0fb7a5f0c25bc0af20b7f605f410678ea335e68c5ce837dfa25d53c6bb14f4

((50 0x9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc 0x4bf5122f344554c53bde2ebb8cd2b7e3d1600ad631c385a5d7cce23c7785459a) (51 0x4e0fb7a5f0c25bc0af20b7f605f410678ea335e68c5ce837dfa25d53c6bb14f4 2000))

 x | o |   
---+---+---
   |   |   
---+---+---
   |   |   


a040e364001629d7026afa01ce3cddab296b4a7e2d9a77025cd4a512699957a134

((50 0xab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0 0xdbc1b4c900ffe48d575b5da5c638040125f65db0fe3e24494b76ea986457d986

## 4. Get values from curried puzzles

In [6]:
board = ['o', ' ', 'x', ' ', 'x', 'x', 'o', ' ', 'o']
board, player = sim_play(board, 'x', 1)

curried_terminate_puzzle = terminate_puzzle.curry(
    0,
    player_one_hash,
    player_two_hash,
    p2_amount
)

curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
    Program.to(board), 
    Program.to(player)
) 

curried_coin_puzzle = coin_puzzle.curry(
    coin_puzzle,
    curried_terminate_puzzle,
    player_one_info,
    player_two_info,
    curried_tic_tac_toe_puzzle)

curried_puzzle = tic_tac_toe.get_curried_puzzle_from_curried_coin_puzzle(curried_coin_puzzle)

print(tic_tac_toe.get_board_from_curried_puzzle(curried_puzzle))
print(tic_tac_toe.get_player_from_curried_puzzle(curried_puzzle))


a0b4b5ba275838828b3d62099b409f6ce0481441c491784a71796bf2fa3298f175

((50 0xab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0 0x4bf5122f344554c53bde2ebb8cd2b7e3d1600ad631c385a5d7cce23c7785459a) (51 0xb4b5ba275838828b3d62099b409f6ce0481441c491784a71796bf2fa3298f175 2000))

 o | x | x 
---+---+---
   | x | x 
---+---+---
 o |   | o 


[111, 120, 120, 32, 120, 120, 111, 32, 111]
o
